Gather necessary pre-requisites:

In [1]:
# import everything you need
from nipype import Node, Workflow
import nipype.interfaces.spm as spm
from nipype.interfaces.base import CommandLine
from nilearn import image as nlimg
from nilearn import masking as nlmask
from nilearn.plotting import plot_anat, plot_epi, plot_roi
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nibabel as nb
import os, re
from itertools import zip_longest

200226-13:56:35,628 nipype.utils INFO:
	 Running nipype version 1.5.0-dev (latest: 1.4.2)


In [2]:
# define useful functions
# define show-slices function
def show_slices(slices):
    """ Function to display row of image slices """
    fig, axes = plt.subplots(1, len(slices))
    for i, slice in enumerate(slices):
        axes[i].imshow(slice.T, cmap="gray", origin="lower")
# add suptitle via plt.suptitle
        
# check size of items and delete them when above a certain threshold
from sys import getsizeof # function is not fully indicative -> look for better option
def check_itemsize(items):
    for i in items:
        # return size of object in bytes
        print("size of item", i, "is:", getsizeof(i))
        if getsizeof(i) > 10000: # think about dynamic coding for this
            del i
            print("deleted, object exceeded set threshold.")
        else:
            print("retained, object is sufficiently small.")

Get files

In [3]:
# set VBM analysis directory
VBM_dir = "/VBM/"

# get normalised GM/WM images to analyse in MultipleRegression model
# change norm_dir to vary between GM and WM analysis
norm_dir="/VBM/DARTEL_norm2mni_c1/"

# grep files according to regex
images_mreg=[i for i in os.listdir(norm_dir) if re.match(r'(smwc.*\d{14,}.*.nii)', i)]

# turn filenames into full-path objects
images_mreg=[os.path.join(norm_dir, i) for i in images_mreg]

In [4]:
#print("Images where normalised only if included in study specific template!")
print(images_mreg, len(images_mreg))

['/VBM/DARTEL_norm2mni_c1/smwc1DEP_FED032_T1_MPRAGE_SAG_ISO_0_9_0005_20150313125246.nii', '/VBM/DARTEL_norm2mni_c1/smwc1CON_FED058_T1_MPRAGE_SAG_ISO_0_9_0005_20160217151251.nii', '/VBM/DARTEL_norm2mni_c1/smwc1CON_FED022_T1_MPRAGE_SAG_ISO_0_9_0005_20141211155503.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_FED011_T1_MPRAGE_SAG_ISO_0_9_0005_20141119161708.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_FED045_T1_MPRAGE_SAG_ISO_0_9_0005_20151105173027.nii', '/VBM/DARTEL_norm2mni_c1/smwc1CON_FED021_T1_MPRAGE_SAG_ISO_0_9_0005_20141211095930.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_FED030_T1_MPRAGE_SAG_ISO_0_9_0005_20150226154844.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_FED052_T1_MPRAGE_SAG_ISO_0_9_0005_20151203172751.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_FED055_T1_MPRAGE_SAG_ISO_0_9_0005_20151214092109.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_FED046_T1_MPRAGE_SAG_ISO_0_9_0005_20151106125904.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_FED029_T1_MPRAGE_SAG_ISO_0_9_0005_20150226091949.nii', '/VBM/DARTEL_norm2mn

In [5]:
# group smwc1/2 images by condition
dep_norm=[]
con_norm=[]

for i in images_mreg:
    match_dep=re.match(r'.*DEP_.*.nii', i)
    match_con=re.match(r'.*CON_.*.nii', i)
    if match_dep:
        dep_norm.append(match_dep.group(0))
    elif match_con:
        con_norm.append(match_con.group(0))
    else:
        print("item in images_mreg does not match regex pattern! Investigate ... ")
        
# sort lists by FED Nr.
dep_norm = sorted(dep_norm)
con_norm = sorted(con_norm)

imreg_norm = dep_norm + con_norm

In [6]:
# control resulting lists and their length
print(dep_norm, len(dep_norm))
print(con_norm, len(con_norm))

print(imreg_norm, len(imreg_norm))

['/VBM/DARTEL_norm2mni_c1/smwc1DEP_FED011_T1_MPRAGE_SAG_ISO_0_9_0005_20141119161708.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_FED014_T1_MPRAGE_SAG_ISO_0_9_0005_20141125153216.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_FED018_T1_MPRAGE_SAG_ISO_0_9_0005_20141208163153.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_FED027_T1_MPRAGE_SAG_ISO_0_9_0005_20150218162601.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_FED028_T1_MPRAGE_SAG_ISO_0_9_0005_20150219142148.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_FED029_T1_MPRAGE_SAG_ISO_0_9_0005_20150226091949.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_FED030_T1_MPRAGE_SAG_ISO_0_9_0005_20150226154844.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_FED031_T1_MPRAGE_SAG_ISO_0_9_0005_20150310152650.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_FED032_T1_MPRAGE_SAG_ISO_0_9_0005_20150313125246.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_FED033_T1_MPRAGE_SAG_ISO_0_9_0005_20150326151309.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_FED034_T1_MPRAGE_SAG_ISO_0_9_0005_20150909125805.nii', '/VBM/DARTEL_norm2mn

Calculate Global Values (used for relating each subject's GM/WM/CSF volume to the total intracranial volume)

In [7]:
# calculate "global values" via nibabel objects: sum up the values and multiply by volume of each voxel
# from: total volume GM, WM and CSF

# find all the files starting with "c1,c2,c3" in all subdirs of the VBM directory via CommandLine interface from nipype
c1_files=CommandLine('find', args='/VBM/FED* -maxdepth 2 -type f -name c1*.nii', terminal_output='allatonce')
c1=c1_files.run()
c2_files=CommandLine('find', args='/VBM/FED* -maxdepth 2 -type f -name c2*.nii', terminal_output='allatonce')
c2=c2_files.run()
c3_files=CommandLine('find', args='/VBM/FED* -maxdepth 2 -type f -name c3*.nii', terminal_output='allatonce')
c3=c3_files.run()

# read the output (find stdout) line by line to list object
globals_GM = c1.runtime.stdout.splitlines()
globals_WM = c2.runtime.stdout.splitlines()
globals_CSF = c3.runtime.stdout.splitlines()

# remove subject 10 from lists(-> was not normalised or in template)
globals_GM = [i for i in globals_GM if i.split('/',3)[2] != "FED010"]
globals_WM = [i for i in globals_WM if i.split('/',3)[2] != "FED010"]
globals_CSF = [i for i in globals_CSF if i.split('/',3)[2] != "FED010"]

# sort globals lists according to subject sequence in imreg_norm
globals_GM_sorted = []
globals_WM_sorted = []
globals_CSF_sorted = []

for i in imreg_norm:
    for j in globals_GM:
        if i.split('_',4)[3] == j.split('/',3)[2]:
            globals_GM_sorted.append(j)
    for k in globals_WM:
        if i.split('_',4)[3] == k.split('/',3)[2]:
            globals_WM_sorted.append(k)
    for l in globals_CSF:
        if i.split('_',4)[3] == l.split('/',3)[2]:
            globals_CSF_sorted.append(l)

In [8]:
# control results
print(globals_GM_sorted[0:4], len(globals_GM_sorted))
print(globals_WM_sorted[0:4], len(globals_WM_sorted))
print(globals_CSF_sorted[0:4], len(globals_CSF_sorted))
print(imreg_norm[0:4], len(imreg_norm))

['/VBM/FED011/DARTEL_newsegment/c1DEP_FED011_T1_MPRAGE_SAG_ISO_0_9_0005_20141119161708.nii', '/VBM/FED014/DARTEL_newsegment/c1DEP_FED014_T1_MPRAGE_SAG_ISO_0_9_0005_20141125153216.nii', '/VBM/FED018/DARTEL_newsegment/c1DEP_FED018_T1_MPRAGE_SAG_ISO_0_9_0005_20141208163153.nii', '/VBM/FED027/DARTEL_newsegment/c1DEP_FED027_T1_MPRAGE_SAG_ISO_0_9_0005_20150218162601.nii'] 62
['/VBM/FED011/DARTEL_newsegment/c2DEP_FED011_T1_MPRAGE_SAG_ISO_0_9_0005_20141119161708.nii', '/VBM/FED014/DARTEL_newsegment/c2DEP_FED014_T1_MPRAGE_SAG_ISO_0_9_0005_20141125153216.nii', '/VBM/FED018/DARTEL_newsegment/c2DEP_FED018_T1_MPRAGE_SAG_ISO_0_9_0005_20141208163153.nii', '/VBM/FED027/DARTEL_newsegment/c2DEP_FED027_T1_MPRAGE_SAG_ISO_0_9_0005_20150218162601.nii'] 62
['/VBM/FED011/DARTEL_newsegment/c3DEP_FED011_T1_MPRAGE_SAG_ISO_0_9_0005_20141119161708.nii', '/VBM/FED014/DARTEL_newsegment/c3DEP_FED014_T1_MPRAGE_SAG_ISO_0_9_0005_20141125153216.nii', '/VBM/FED018/DARTEL_newsegment/c3DEP_FED018_T1_MPRAGE_SAG_ISO_0_9_0005_

In [9]:
# create nibabel objects from all files
globals_GM_nb=[nb.load(i) for i in globals_GM_sorted]
globals_WM_nb=[nb.load(i) for i in globals_WM_sorted]
globals_CSF_nb=[nb.load(i) for i in globals_CSF_sorted]

# create list with the func data via dataobj property (avoids caching entire object (.fdata()) but gets func data as well)
globals_GM_fvals=[i.dataobj for i in globals_GM_nb]
globals_WM_fvals=[i.dataobj for i in globals_WM_nb]
globals_CSF_fvals=[i.dataobj for i in globals_CSF_nb]

In [10]:
# 0 values outside the brain in each volume -> 0 vals overbias and make globals smaller
# c1,2,3 also are not normalised -> different positions for different subjects
# -> take all c images and calculate below values for each matter ;)

# re-asign lists
globals_GM=[]
globals_WM=[]
globals_CSF=[]

# loop over GM WM CSF lists and calculate globals
for a,b in zip_longest(globals_GM_fvals,globals_GM_nb):
    globals_GM.append((np.sum(a[:,:,:][a[:,:,:] != 0])
                       * np.product(b.header['pixdim'][1:4]))
                       / np.size(a[:,:,:][a[:,:,:] != 0]))

for a,b in zip_longest(globals_WM_fvals,globals_WM_nb):
    globals_WM.append((np.sum(a[:,:,:][a[:,:,:] != 0])
                       * np.product(b.header['pixdim'][1:4]))
                       / np.size(a[:,:,:][a[:,:,:] != 0]))

for a,b in zip_longest(globals_CSF_fvals,globals_CSF_nb):
    globals_CSF.append((np.sum(a[:,:,:][a[:,:,:] != 0])
                        * np.product(b.header['pixdim'][1:4]))
                        / np.size(a[:,:,:][a[:,:,:] != 0]))

# combine globals to get total intracranial volume (TiCV)
globals_TiCV=[]

for a,b,c in zip_longest(globals_GM,globals_WM,globals_CSF):
    globals_TiCV.append(np.sum([a,b,c]))

In [11]:
# establish range of TiCV globals
print("range of globals_TiCV is:     ", np.min(globals_TiCV), " - ", np.max(globals_TiCV))
# establish range of values in c files
#for a,b,c in zip_longest(globals_GM_fvals, globals_WM_fvals, globals_CSF_fvals):
#    print(np.amin(a), np.amax(a),np.amin(b), np.amax(b),np.amin(c), np.amax(c))
print("range of values in c images is always the same:", np.mean([np.amin(i) for i in globals_CSF_fvals]),
      " - ", np.mean([np.amax(i) for i in globals_CSF_fvals]))

range of globals_TiCV is:      1.1573917414830786  -  1.4875706951988656
range of values in c images is always the same: 0.0  -  1.0000000591389835


Get all necessary Covariates (Condition and Nuisance)

In [12]:
# create covariates (SPM makes NO DIFFERENCE btw covariates and conditions)
# get depression, sex and age - data from list in .xlsx file 
cov_file_ID="FED_Subject_Assignments.xlsx"
content=pd.read_excel(cov_file_ID, sheet_name="Sheet2", usecols=['Sub Num FED_XXX', 'BDI 22 Score', 'Gender', 'Age'])

# remove subject 10 from lists(-> was not normalised or in template)
content=content[content["Sub Num FED_XXX"] != 10]

# sort panda dataframe according to subject sequence in imreg_norm
# define sort-by list (get FED_ID and format to fit entries in dataframe)
mreg_ID = [i.split('_',4)[3][3:].lstrip("0") for i in imreg_norm]
# transform values to integers
mreg_ID = [np.int(i) for i in mreg_ID]
# define a categorical variable to sort a column and corresponding lines after
content['mreg_ID'] = pd.Categorical(content['Sub Num FED_XXX'], categories = mreg_ID, ordered = True)
# sort dataframe
content.sort_values('mreg_ID', inplace=True)

# get relevant covariates
dep=content['BDI 22 Score'].tolist()
sex=content['Gender'].tolist()
age=content['Age'].tolist()

# depression dep: 1, con: 0
cov_dep={"vector":dep, "name":"MDD", "centering":0}

# sex - dictionary {vector, name, centering} female: 1, male: 0
cov_sex={"vector":sex, "name":"Sex", "centering":0}

# age - dictionary {vector, name, centering}
cov_age={"vector":age, "name":"Age", "centering":0}

In [13]:
# control results
print(mreg_ID)
print(content)

[11, 14, 18, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 51, 52, 55, 57, 59, 66, 6, 7, 8, 9, 12, 13, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 49, 50, 53, 54, 56, 58, 60, 61, 62, 63, 64, 65, 67, 68]
    Sub Num FED_XXX  Gender  Age  BDI 22 Score mreg_ID
5                11       0   18             1      11
8                14       1   18             1      14
12               18       1   18             1      18
21               27       1   19             1      27
22               28       1   20             1      28
..              ...     ...  ...           ...     ...
57               63       0   19             0      63
58               64       1   19             0      64
59               65       1   18             0      65
61               67       1   20             0      67
62               68       1   18             0      68

[62 rows x 5 columns]


In [15]:
# control input length of all mreg inputs
print(len(imreg_norm),
     len(globals_TiCV),
     len(cov_dep['vector']),
     len(cov_sex['vector']),
     len(cov_age['vector']))

62 62 62 62 62


Write and Estimate Your Model

In [16]:
# create multiple regression Node
mreg = Node(spm.MultipleRegressionDesign(), name="SPM_MultiReg")
mreg.base_dir= VBM_dir
mreg.inputs.in_files = imreg_norm
mreg.inputs.global_calc_values = globals_TiCV
# use proportional global normalization
mreg.inputs.global_normalization = 2
# cut-off value for a-posteriori probability of GM at a given voxel: 5%, [maybe 20%? -> Ashburner VBM Tutorial]
mreg.inputs.threshold_mask_absolute = 0.05
# think of mask image to threshold
#mreg.inputs.explicit_mask_file=
mreg.inputs.covariates = [cov_dep, cov_sex, cov_age]

In [15]:
# make model (design, not estimate) and print results
results = mreg.run()
print(results.outputs)

200225-15:03:31,665 nipype.workflow INFO:
	 [Node] Setting-up "MultipleRegression" in "/VBM/MultipleRegression".
200225-15:03:31,680 nipype.workflow INFO:
	 [Node] Running "MultipleRegression" ("nipype.interfaces.spm.model.MultipleRegressionDesign")
200225-15:03:39,908 nipype.workflow INFO:
	 [Node] Finished "MultipleRegression".

spm_mat_file = /VBM/MultipleRegression/SPM.mat



In [16]:
# specify MReg directory
MReg_dir = mreg.base_dir + mreg.name

# estimate model
est_mreg = Node(spm.EstimateModel(), name="estimate_MReg")
est_mreg.base_dir = MReg_dir
est_mreg.inputs.spm_mat_file = MReg_dir+"SPM.mat"
est_mreg.inputs.estimation_method = {"Classical":1}

In [17]:
# run estimation and print output
results = est_mreg.run()
print(results.outputs)

200225-15:03:46,0 nipype.workflow INFO:
	 [Node] Setting-up "estimate_MReg" in "/VBM/MultipleRegression/estimate_MReg".
200225-15:03:46,3 nipype.workflow INFO:
	 [Node] Running "estimate_MReg" ("nipype.interfaces.spm.model.EstimateModel")
200225-15:04:11,60 nipype.workflow INFO:
	 [Node] Finished "estimate_MReg".

ARcoef = <undefined>
Cbetas = <undefined>
RPVimage = /VBM/MultipleRegression/estimate_MReg/RPV.nii
SDbetas = <undefined>
SDerror = <undefined>
beta_images = ['/VBM/MultipleRegression/estimate_MReg/beta_0001.nii', '/VBM/MultipleRegression/estimate_MReg/beta_0002.nii', '/VBM/MultipleRegression/estimate_MReg/beta_0003.nii', '/VBM/MultipleRegression/estimate_MReg/beta_0004.nii']
labels = <undefined>
mask_image = /VBM/MultipleRegression/estimate_MReg/mask.nii
residual_image = /VBM/MultipleRegression/estimate_MReg/ResMS.nii
residual_images = <undefined>
spm_mat_file = /VBM/MultipleRegression/estimate_MReg/SPM.mat



Write and Estimate Contrasts to run for the Model above

In [18]:
# specify MReg directory
MReg_dir = est_mreg.base_dir + est_mreg.name

# collect beta images from Model estimation
beta_img = sorted([i for i in os.listdir(MReg_dir) if re.match(r'^beta.*.nii', i)])
# turn beta filenames into full-path objects
beta_img = [os.path.join(MReg_dir, i) for i in beta_img]

# define contrasts
cont1 = ('Dep > Con', 'T', ['mean','MDD','Sex','Age'], [0, 1, 0, 0])
cont2 = ('Male > Female', 'T', ['mean','MDD','Sex','Age'], [0, 0, -1, 0])
cont3 = ('Young > Old', 'T', ['mean','MDD','Sex','Age'], [0, 0, 0, -1])
contrasts = [cont1,cont2,cont3]

# estimate contrast
est_contrasts = Node(spm.EstimateContrast(), name="estimate_Contrats")
est_contrasts.base_dir = MReg_dir
est_contrasts.inputs.spm_mat_file = MReg_dir+"SPM.mat"
est_contrasts.inputs.residual_image = MReg_dir+"ResMS.nii"
est_contrasts.inputs.beta_images = beta_img
est_contrasts.inputs.contrasts = contrasts
# clarify this as 2nd level contrast
est_contrasts.inputs.group_contrast = True

In [19]:
# run estimation and print output
results = est_contrasts.run() 
print(results.outputs)

200225-15:05:43,587 nipype.workflow INFO:
	 [Node] Setting-up "estimate_Contrats" in "/VBM/MultipleRegression/estimate_MReg/estimate_Contrats".
200225-15:05:43,598 nipype.workflow INFO:
	 [Node] Running "estimate_Contrats" ("nipype.interfaces.spm.model.EstimateContrast")
200225-15:05:51,817 nipype.workflow INFO:
	 [Node] Finished "estimate_Contrats".

con_images = ['/VBM/MultipleRegression/estimate_MReg/estimate_Contrats/con_0001.nii', '/VBM/MultipleRegression/estimate_MReg/estimate_Contrats/con_0002.nii', '/VBM/MultipleRegression/estimate_MReg/estimate_Contrats/con_0003.nii']
ess_images = <undefined>
spmF_images = <undefined>
spmT_images = ['/VBM/MultipleRegression/estimate_MReg/estimate_Contrats/spmT_0001.nii', '/VBM/MultipleRegression/estimate_MReg/estimate_Contrats/spmT_0002.nii', '/VBM/MultipleRegression/estimate_MReg/estimate_Contrats/spmT_0003.nii']
spm_mat_file = /VBM/MultipleRegression/estimate_MReg/estimate_Contrats/SPM.mat



Threshold statistics and plot results

In [50]:
# define statistics directory
stats_dir = est_contrasts.base_dir + est_contrasts.name
stat_images = sorted([os.path.join(stats_dir, i) for i in os.listdir(stats_dir) if re.match(r'(^spmT_)', i)])

# apply FDR correction to statistical contrasts
# adapt index to matlab index
for i in range(1, len(stat_images)+1):
    FDR_thresh = Node(spm.Threshold(), name="FDR_spmT" + str(i))
    FDR_thresh.base_dir = stats_dir
    FDR_thresh.inputs.spm_mat_file = stats_dir + '/SPM.mat'
    FDR_thresh.inputs.contrast_index = i
    FDR_thresh.inputs.stat_image = stat_images[i-1] # need python indexing here
    FDR_thresh.inputs.extent_fdr_p_threshold = 0.05
    # minimal cluster size in voxels
    #FDR_thresh.inputs.extent_threshold =
    # default: 0.05; for initial thresholding (defining clusters)
    #FDR_thresh.inputs.height_threshold = 
    # default: p-value; p-value or stat
    FDR_thresh.inputs.height_threshold_type = "p-value"
    # run thresholding and print output
    results = FDR_thresh.run() 
    print(results.outputs)

200225-16:19:30,675 nipype.workflow INFO:
	 [Node] Setting-up "FDR_spmT1" in "/VBM/MultipleRegression/estimate_MReg/estimate_Contrats/FDR_spmT1".
200225-16:19:30,681 nipype.workflow INFO:
	 [Node] Running "FDR_spmT1" ("nipype.interfaces.spm.model.Threshold")
200225-16:19:33,664 nipype.workflow INFO:
	 [Node] Finished "FDR_spmT1".

activation_forced = False
cluster_forming_thr = 5.038015
n_clusters = 0
pre_topo_fdr_map = /VBM/MultipleRegression/estimate_MReg/estimate_Contrats/FDR_spmT1/spmT_0001_pre_topo_thr.nii
pre_topo_n_clusters = 0
thresholded_map = /VBM/MultipleRegression/estimate_MReg/estimate_Contrats/FDR_spmT1/spmT_0001_thr.nii

200225-16:19:33,668 nipype.workflow INFO:
	 [Node] Setting-up "FDR_spmT2" in "/VBM/MultipleRegression/estimate_MReg/estimate_Contrats/FDR_spmT2".
200225-16:19:33,676 nipype.workflow INFO:
	 [Node] Running "FDR_spmT2" ("nipype.interfaces.spm.model.Threshold")
200225-16:19:36,594 nipype.workflow INFO:
	 [Node] Finished "FDR_spmT2".

activation_forced = Fal

In [ ]:
# prepare plotting of SPM12 stats results



Testing Zone:

In [107]:

# get original T1 images for all FEDs
struct_DEP=CommandLine('ls', args='/VBM/FED*/DEP*T1*.nii', terminal_output='allatonce')
structD=struct_DEP.run()
struct_CON=CommandLine('ls', args='/VBM/FED*/CON*T1*.nii', terminal_output='allatonce')
structC=struct_CON.run()
# read the output (find stdout) line by line to list object
struct_all = structD.runtime.stdout.splitlines() + structC.runtime.stdout.splitlines()
struct_all = []
# calculate mean image

# save mean image to VBM directory


TypeError: __init__() takes 1 positional argument but 2 were given

In [35]:
# apply full set of thresholds from SPM12
thresh_all = Node(, name="SPM_thresholds")
thresh_all.base_dir = stats_dir
thresh_all.inputs.spm_mat_file = stats_dir + '/SPM.mat'
thresh_all.inputs.contrast_index = 1
thresh_all.inputs.stat_image = stat_images[0]
thresh_all.inputs.height_threshold = 4.5
# minimal cluster size in voxels
#thresh_all.inputs.extent_threshold =

NameError: name 'ThresholdStatistics' is not defined

In [ ]:
# run thresholding and print output
results = thresh_all.run() 
print(results.outputs)